# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### Madre - Lazio

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187791-d10598456-Reviews-Madre-Rome_Lazio.html

Home --> Ristoranti --> Firenze --> Best_ranking--> SENZ_ALTRO_Bistrot

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187791-d10598456-Reviews-Madre-Rome_Lazio.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
from tqdm import tqdm

URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187791-d10598456-Reviews-or10-Madre-Rome_Lazio.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187791-d10598456-Reviews-or'
URL_p2 = '0-Madre-Rome_Lazio.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in tqdm(range(1,total_pages)):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_mobile))

100%|██████████| 65/65 [01:27<00:00,  1.29s/it]

Executed
total_pages:  66
Nomi:  656
Commenti:  656
Voti:  656
Scritto da:  656


In [0]:
# display(all_reviews)

In [0]:
# print(all_names)

In [0]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [0]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [9]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 656 
 Recensioni analizzabili: 232 
 Recensioni scartate: 424 



['Location ok molto particolare ma per il resto niente di che ..la pizza forse buona ma non buonissima ..per quello che si spende non ne vale la pena .',
 'RECENSIONE_TROPPO_LUNGA',
 'Location molto bella e curata nel centro storico di Roma.\nSono stata in questo posto con amici a pranzo, per cui il locale non era affollato.\nPersonale molto attento e disponibile.\nCibo buono.\nPrezzi non eccessivi.',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'Visitato una volta e ritornato: volte si a volte no, qualcosa si qualcosa no. Conto superiore a quanto ci aspettavamo.',
 'RECENSIONE_TROPPO_LUNGA']

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [10]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['Flyer739413', 'LAURA D', 'rominak785', 'O4700KIlorenzos', 'ESMERALDA28', 'PaoloRiccardoCarrara', 'my_little_snail', 'Peppe F', 'corrado51lecce', 'Sightseer71500', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'ilariam789', 'Giuliana1968', 'Martina1294', 'Michela P', 'Aenne76', 'SwaMI89', 'Manugiorgia', 'catia l', 'merenderocarl', 'Patrizia S', 'Roam770587', 'O3599ZKsaram', 'Carloalvise', 'DeCichis', 'corrado51lecce', 'J1144ZBrobertoc', 'andreatM8525KM', 'gionniv445', '583biagioc', 'W9057CZrobertob', 'emaloli94', 'DoraEmanuele', 'D3537XGclaudiac', 'MartyeFra', 'luca2013', 'filippa83', 'E7353VMsarac', 'priscilla_collacchi', 'Andrea196666', 'cnssra', 'erduce', 'Francesco S', 'Barbara D', 'DanielePascale_922', 'RobertoR67', 'newrita', 'Lucia Q', 'lauragb52', 'Mauro F', 'misscappy11', 'federicospruz', 'LiviaA90', 'P8329KEriccardos', 'prl7', 'ipzs91', 'fradalessa', 'Z9008IHbarbarac', 'Lanfranco R', 'cocchic', '708edoardom', 'c

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [11]:
!pip install gender_guesser

In [12]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'female', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'female', 

In [13]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'M', 'UNKNOWN', 'F', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'U

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [14]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  656
Commenti:  656
Voti:  656
Mobile:  656


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [15]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  40


[['Giuliana',
  'F',
  'La location è splendida, quanto inaspettata.\ni prodotti freschi e di prima qualità.\nIl personale attento alle esigenze del cliente, sono sempre gentili e sorridenti\nlo consiglio vivamente!!!',
  '4',
  'PC'],
 ['Martina',
  'F',
  'Sono stata due giorni fa a cena da Madre con il mio fidanzato e siamo rimasti davvero soddisfati. Location veramente bella, curata e accogliente. Personale preparatissimo e attento. Cibo particolare ma tutto buonissimo. Ci tornerò sicuramente !',
  '5',
  'Mobile'],
 ['Aenne',
  'F',
  "Una vera boccata d'aria fresca per Roma.\nBella location, ottimi drink, buon cibo.\nPrezzi un tantino altini ma decisamente raccomandabile!",
  '4',
  'PC'],
 ['Andrea',
  'F',
  'Consigliatissimo!!!\nSi mangia molto bene, si beve benissimo e se vi servirà Fabio sarete super coccolati! Bravo Fabio e bellissimo posto!!! Torneremo sicuramente!',
  '5',
  'Mobile'],
 ['Barbara',
  'F',
  'Ottima location in pieno centro di Roma giardino fantastico, cib

### Creiamo la tabella

In [0]:
# !pip install pandas
# !pip install xlsxwriter
import pandas as pd

In [0]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_17.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [18]:
partial_dataframe = pd.read_excel("RISTORANTE_17.xlsx")

display(partial_dataframe)

,Genere,Nomi,Recensioni,Scritta da,Voti
0,F,Giuliana,"La location è splendida, quanto inaspettata.\n...",PC,4
1,F,Martina,Sono stata due giorni fa a cena da Madre con i...,Mobile,5
2,F,Aenne,Una vera boccata d'aria fresca per Roma.\nBell...,PC,4
3,F,Andrea,"Consigliatissimo!!!\nSi mangia molto bene, si ...",Mobile,5
4,F,Barbara,Ottima location in pieno centro di Roma giardi...,Mobile,5
5,F,Lucia,"Bellissimo giardino fresco,ordinato.Personale ...",Mobile,5
6,M,Mauro,In uno dei quartieri più belli di Roma....ques...,Mobile,4
7,M,Samuele,Ottimo posto personale bellissimo ed educato c...,Mobile,5
8,F,Elena,Location splendida che riproduce una veranda f...,Mobile,4
9,F,Valentina,"Sempre impeccabili, la cucina è favolosa, i ma...",PC,5
